In [ ]:
!pip install openai

In [ ]:
# Standard library imports
import os
import csv
import json

# Third-party library imports
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
# Set up the OpenAI client
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

def load_dataset_from_csv(file_path):
    """
    Load a dataset from a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        list: A list of dictionaries containing 'label' and 'sentence'.
    """
    dataset = []
    with open(file_path, 'r', encoding='cp1252') as file:
        reader = csv.reader(file, delimiter='|')
        for row in reader:
            if len(row) == 2:
                label, sentence = row
                dataset.append({'label': label.strip(), 'sentence': sentence.strip()})
    print(f"Carregadas {len(dataset)} linhas de {file_path}")
    return dataset

class Sentence(BaseModel):
    """
    A Pydantic model representing a sentence with alternative texts.
    """
    alternative_text: list[str]

def generate_alternative_sentences(intent, sentence, n_augmentations):
    """
    Generate alternative sentences for data augmentation.

    Args:
        intent (str): The intent label of the sentence.
        sentence (str): The original sentence.
        n_augmentations (int): The number of augmentations to generate.

    Returns:
        list: A list of alternative sentences.
    """
    prompt = (
        f"Gere {n_augmentations} versões diversas da seguinte frase com a intenção '{intent}':\n"
        f"{sentence}\n\n"
        "Instruções:\n"
        "- Escreva como um brasileiro.\n"
        "- Use abreviações às vezes.\n"
        "- Varie os estilos linguísticos, alternando entre formal e informal aleatoriamente mas não use gíricas como 'ixi', 'véi', entre outras do gênero.\n"
        "- A finalidade é fazer data augmentation para um modelo de linguagem."
        "- A frase não deve ser parecida. Você deve captar a intenção do usuário e escrever uma frase variante."
    )
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "Você é um assistente útil."},
            {"role": "user", "content": prompt}
        ],
        response_format=Sentence,
    )
    event = completion.choices[0].message.parsed
    print(f"Frase original: {sentence}")
    print(f"Frase gerada: {event.alternative_text}")
    return event.alternative_text

def main():
    """
    Main function to load the original dataset, generate augmented sentences, and save the augmented dataset.
    """
    input_file = 'original_dataset.csv'
    output_file = 'augmented_dataset.json'
    target_sentence_count = 300  # Target number of sentences per intent

    # Load original dataset from CSV
    dataset = load_dataset_from_csv(input_file)

    intents = {}

    # Group sentences by label
    for item in dataset:
        label = item['label']
        sentence = item['sentence']
        if label not in intents:
            intents[label] = []
        intents[label].append(sentence)

    augmented_dataset = []

    for intent, sentences in intents.items():
        current_sentence_count = len(sentences)
        additional_sentences_needed = target_sentence_count - current_sentence_count

        print(f"Gerando aumentações para o rótulo: {intent}, frases atuais: {current_sentence_count}, aumentações necessárias: {additional_sentences_needed}")

        while additional_sentences_needed > 0:
            for sentence in sentences:
                n_augmentations = min(additional_sentences_needed, 5)  # Limite de 5 para evitar muitas chamadas
                augmented_sentences = generate_alternative_sentences(intent, sentence, n_augmentations)
                for aug_sentence in augmented_sentences:
                    if additional_sentences_needed <= 0:
                        break
                    augmented_dataset.append({'label': intent, 'sentence': aug_sentence})
                    additional_sentences_needed -= 1
                if additional_sentences_needed <= 0:
                    break

        # Adicionar as sentenças originais ao dataset aumentado
        for sentence in sentences:
            augmented_dataset.append({'label': intent, 'sentence': sentence})

    # Save the augmented dataset to a JSON file
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(augmented_dataset, file, ensure_ascii=False, indent=4)

    print(f"Dataset aumentado salvo em {output_file}")

if __name__ == "__main__":
    main()

Carregadas 428 linhas de original_dataset.csv
Gerando aumentações para o rótulo: Acesso a conta, frases atuais: 16, aumentações necessárias: 284
Frase original: Atualizei os site e agora como acessar minha conta
Frase gerada: ['Fiz mudanças no site e gostaria de saber como faço para acessar minha conta agora.', 'Alterei o site recentemente, alguém poderia me informar como entro na minha conta?', 'Arrumei o site, mas tô meio perdido pra acessar minha conta, alguém ajuda?', 'Modifiquei o layout do site e preciso de instruções para acessar minha conta atual.', 'O site tá diferente. Alguém sabe me dizer como faço pra acessar minha conta agora?']
Frase original: Boa tarde estou tendo problema com o pin\neu não lembro a senha do Pin ,e por isso não consigo trocar a senha do pin e nem add beneficiário
Frase gerada: ['Boa tarde, estou tendo dificuldades com o código PIN e não consigo lembrar a senha, o que impede a troca do PIN ou a adição de novos beneficiários.', 'Olá, estou com problema no 

In [ ]:
# Nome do arquivo JSON
file_name = 'augmented_dataset.json'

# Ler o arquivo JSON
with open(file_name, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Converter para DataFrame
df = pd.DataFrame(data)

# Salvar como CSV separado por |
df.to_csv('output.csv', sep='|', index=False)

# Mostrar o DataFrame
print(df)

                           label  \
0                 Acesso a conta   
1                 Acesso a conta   
2                 Acesso a conta   
3                 Acesso a conta   
4                 Acesso a conta   
...                          ...   
5395  Tempo de entrega do cartao   
5396  Tempo de entrega do cartao   
5397  Tempo de entrega do cartao   
5398  Tempo de entrega do cartao   
5399  Tempo de entrega do cartao   

                                               sentence  
0     Fiz mudanças no site e gostaria de saber como ...  
1     Alterei o site recentemente, alguém poderia me...  
2     Arrumei o site, mas tô meio perdido pra acessa...  
3     Modifiquei o layout do site e preciso de instr...  
4     O site tá diferente. Alguém sabe me dizer como...  
...                                                 ...  
5395         Obrigada\nTem previsão de chegar o cartão?  
5396     Ola\nEm quantos dia chega o cartão de remessa?  
5397      Olá\nmeu cartão laranja eu não rece